In [5]:
import sys
sys.path.append('../../')
from search import search_local_distance
import numpy as np
from tools.tools import *
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

Get gaia catalogue distance and distance error

In [6]:
samples = search_local_distance.get_entire_catalogue()
dist, dist_err = samples.T

Define parammeters for making histogram. All units in kpc.

In [7]:
d_pixel = 0.001
d_max = 6
d_min = 0
bins = int((d_max - d_min)/d_pixel)
epsilon = 0.2
bins_close = int((epsilon - d_min)/d_pixel)

Get stars within epsilon kpc around sun as well as their error.

In [8]:
mask = dist < epsilon
dist_close = dist[mask]
dist_close_err = dist_err[mask]

(366209,) (366209,)


Graph 1D histogram for distribution of distance for all of gaia catalogue

In [ ]:
histogram_gaia, edges, patches = plt.hist(dist_coord, bins = bins_d, range = (d_min, d_max))

Graph 1D histogram for distribution of distance for all stars in epsilon region

In [ ]:
histogram_close, edges_close, patches_close = plt.hist(dist_close, bins = bins_close)

Define a function that get the gaia spherical number density function from histogram result

In [ ]:
def n_gaia(d, histogram):
    d_index = ((d-d_min)/d_pixel).astype(int)-1
    result = []
    for i in range(np.size(d)):
        result.append(histogram[d_index[i]])
    return np.array(result)

Define the true sperhical number density function

In [ ]:
def n(d, n_0):
    return n_0*4*(np.pi)*(d**2)

Curve fit spherical density with stars in epsilon region.

In [ ]:
popt, pcov = curve_fit(n_spherical, dist_close, n_gaia(dist_close, histogram_close))
n_0 = popt[0]
std_n_0 = pcov[0][0]
print("optimized n_0 =", n_0)
print("standard deviation of n_0 =", std_n_0)

Plot theoretical prediction over histogram in epsilon neighborhood.

In [ ]:
plt.hist(dist_close, bins = bins_close)
d_linspace = np.linspace(d_min, epsilon, bins_close, endpoint = False)
plt.plot(d_linspace, n(d_linspace, n_0))

Plot theoretical prediction over real histogram for all stars.

In [ ]:
plt.hist(dist_coord, bins = bins_d, range = (d_min, d_max))
plt.xlim(0,0.5)
d_linspace = np.linspace(d_min, epsilon, bins, endpoint = False)
plt.plot(d_linspace, n(d_linspace, n_0))

Calculate the Pearson Chi Square statistic